In [186]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns

from datetime import datetime

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, roc_curve, auc, confusion_matrix
from imblearn.over_sampling import ADASYN
from imblearn.over_sampling import SMOTE
from scipy.stats import zscore
from scipy.stats import chi2_contingency


import pandas as pd
import numpy as np

#Graphiques
import seaborn as sns
sns.set()
import plotly.express as px
import plotly.graph_objects as gp
import matplotlib.pyplot as plt

#Statistiques
from scipy.stats import chi2_contingency

# Machine Learning
from sklearn.cluster import KMeans
import sklearn.preprocessing as preproc
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import KFold
from sklearn import metrics
from sklearn.model_selection import cross_val_score

from sklearn.metrics import (
    confusion_matrix,
    ConfusionMatrixDisplay,
    classification_report,
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    roc_curve
)


In [144]:
input_path = "./01_Input_Data"
output_path = "./2_outputs"
df = pd.read_csv(input_path + '/bank-additional-full.csv', delimiter = ';')

# Visualisation des données



In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.duplicated().sum()

In [149]:
def afficher_distribution(df, colonne):
    # Création de la figure et des sous-graphiques
    fig, ax = plt.subplots(figsize=(12, 6))

    # Tracer l'histogramme avec la courbe de densité
    sns.histplot(df[colonne], kde=True, bins=25, stat='density', linewidth=0, ax=ax)

    # Titre et labels
    ax.set_title(f'Distribution de la variable: {colonne}', fontsize=15)
    ax.set_xlabel(colonne, fontsize=12)
    ax.set_ylabel('Densité', fontsize=12)

    # Calcul des statistiques descriptives
    stats = df[colonne].describe()

    # Texte contenant les statistiques descriptives
    stats_text = "\n".join([f"{key}: {value:.2f}" for key, value in stats.items()])

    # Ajouter le texte à côté du graphique
    plt.gcf().text(0.75, 0.5, stats_text, fontsize=12, bbox=dict(facecolor='white', alpha=0.8))

    # Afficher le graphique
    plt.tight_layout()
    plt.show()

    
def afficher_distribution_categorielle(df, colonne):
    plt.figure(figsize=(10, 6))

    sns.countplot(data=df, x=colonne)

    plt.title(f'Distribution des catégories de : {colonne}', fontsize=15)
    plt.xlabel(colonne, fontsize=12)
    plt.ylabel('Nombre d\'occurrences', fontsize=12)

    plt.xticks(rotation=45)
    
    plt.show()


In [ ]:
afficher_distribution(df, 'age')


In [ ]:
afficher_distribution_categorielle(df, 'job')

In [ ]:
afficher_distribution_categorielle(df, 'marital')

In [ ]:
afficher_distribution_categorielle(df, 'education')

In [ ]:
afficher_distribution_categorielle(df, 'default')

In [ ]:
afficher_distribution_categorielle(df, 'housing')

In [ ]:
afficher_distribution_categorielle(df, 'loan')

In [ ]:
afficher_distribution_categorielle(df, 'contact')

In [ ]:
afficher_distribution_categorielle(df, 'month')

In [ ]:
afficher_distribution_categorielle(df, 'day_of_week')

In [ ]:
afficher_distribution(df, 'duration')

In [ ]:
afficher_distribution(df, 'campaign')

In [ ]:
afficher_distribution(df, 'pdays')

In [ ]:
afficher_distribution(df, 'previous')

In [ ]:
afficher_distribution_categorielle(df, 'poutcome')

In [ ]:
afficher_distribution(df, 'emp.var.rate')

In [ ]:
afficher_distribution(df, 'cons.price.idx')

In [ ]:
afficher_distribution(df, 'cons.conf.idx')

In [ ]:
afficher_distribution(df, 'euribor3m')

In [ ]:
afficher_distribution(df, 'nr.employed')

In [ ]:
afficher_distribution_categorielle(df, 'y')

In [171]:
def plot_numeric_correlation(df):
    """
    Cette fonction affiche une matrice de corrélation pour les variables numériques d'un DataFrame.
    """
    # Sélectionner uniquement les colonnes numériques
    numeric_df = df.select_dtypes(include=['number'])

    # Calcul de la matrice de corrélation
    corr_matrix = numeric_df.corr()

    # Affichage de la matrice de corrélation sous forme de carte thermique
    plt.figure(figsize=(10, 8))
    sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title("Matrice de Corrélation des Variables Numériques")
    plt.show()

def cramers_v(x, y):
    """
    Calcul du V de Cramér pour mesurer l'association entre deux variables catégorielles.
    """
    # Créer une table de contingence entre les deux variables
    contingency_table = pd.crosstab(x, y)

    # Effectuer le test du chi carré
    chi2, p, dof, expected = chi2_contingency(contingency_table)

    # Calcul du V de Cramér
    n = contingency_table.sum().sum()  # Taille totale de l'échantillon
    min_dim = min(contingency_table.shape) - 1  # Minimum entre (k1-1) et (k2-1)
    v_cramer = np.sqrt(chi2 / (n * min_dim))

    return v_cramer

def plot_categorical_association(df):
    """
    Cette fonction affiche une matrice d'association (V de Cramér) pour les variables catégorielles d'un DataFrame.
    """
    # Sélectionner uniquement les colonnes catégorielles
    categorical_df = df.select_dtypes(include=['object', 'category'])

    # Initialiser une matrice vide pour les résultats de V de Cramér
    v_matrix = pd.DataFrame(index=categorical_df.columns, columns=categorical_df.columns)

    # Calculer les valeurs de V de Cramér pour chaque paire de variables
    for col1 in categorical_df.columns:
        for col2 in categorical_df.columns:
            v_matrix.loc[col1, col2] = cramers_v(categorical_df[col1], categorical_df[col2])

    # Affichage de la matrice d'association sous forme de carte thermique
    plt.figure(figsize=(10, 8))
    sns.heatmap(v_matrix.astype(float), annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
    plt.title("Matrice d'Association des Variables Catégorielles (V de Cramér)")
    plt.show()


In [ ]:
plot_numeric_correlation(df)

In [ ]:
plot_categorical_association(df)

# Prétraitement des données

In [175]:
df.drop(columns =['duration'], inplace = True, errors = "ignore")

In [177]:
def apply_pca(df, features, n_components=1):
    # Extraire les variables d'intérêt
    X = df[features]

    # Standardiser les données
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Appliquer l'ACP'
    pca = PCA(n_components=n_components)
    df['PC'] = pca.fit_transform(X_scaled).flatten()

    # Supprimer les variables d'origine
    df.drop(columns=features, inplace=True, errors='ignore')

    return df

In [ ]:
df.drop(columns = ['default'], inplace = True, errors = "ignore")

df.drop(columns = ['pdays'], inplace = True, errors = "ignore")


features = ["euribor3m", "emp.var.rate", "nr.employed"]

df = apply_pca(df, features, n_components=1)

numerical_df = df.select_dtypes(include=[np.number])

z_scores = numerical_df.apply(zscore)

# Ligne avec une colonne avec z-score > 3
outlier_rows = df[(np.abs(z_scores) > 3).any(axis=1)]

print(outlier_rows)

df = df.drop(outlier_rows.index)

In [ ]:
df.head()

# ICI


In [181]:

def classifier_variables(data_set):
    # Initialisation des listes
    variables_na = []
    variables_numeriques = []
    variables_01 = []
    variables_categorielles = []

    # Parcourir les colonnes du dataset
    for colu in data_set.columns:
        # Vérifier si la colonne contient des valeurs NaN
        if data_set[colu].isna().any():
            variables_na.append(colu)
        else:
            # Vérifier si la colonne est de type numérique
            if data_set[colu].dtype in ["int32", "int64", "float32", "float64"]:
                # Vérifier si la colonne est binaire (deux valeurs uniques)
                if len(data_set[colu].unique()) == 2:
                    variables_01.append(colu)
                else:
                    variables_numeriques.append(colu)
            else:
                # Gérer les colonnes non numériques
                if len(data_set[colu].unique()) == 2:
                    variables_01.append(colu)
                else:
                    variables_categorielles.append(colu)

    # Création du dictionnaire des résultats
    resultat = {
        "variables_na": variables_na,
        "variables_numeriques": variables_numeriques,
        "variables_binaires": variables_01,
        "variables_categorielles": variables_categorielles
    }

    return resultat

# Exemple d'utilisation
# variables= classifier_variables(data_set)
# print(variables)
# data_set.tail()

In [187]:

def encoded_categorial(data):
    """
    Effectue le one-hot encoding sur les variables catégoriques.

    Args:
        data (pd.DataFrame): Le DataFrame contenant les données à traiter.
        variables (dict): Dictionnaire contenant les listes de colonnes sous les clés :
                          - 'variables_categorielles' : liste des colonnes catégoriques.
                          - 'variables_binaires' : liste des colonnes binaires.
                          - 'variables_numeriques' : liste des colonnes numériques.

    Returns:
        pd.DataFrame: Un DataFrame avec les variables catégoriques encodées,
                      et les autres colonnes inchangées.
    """
    # Récupérer les variables catégoriques, binaires et numériques
    variables= classifier_variables(data)
    variables_categorielles = variables.get("variables_categorielles", [])
    variables_binaires = variables.get("variables_binaires", [])
    variables_numeriques = variables.get("variables_numeriques", [])


    # Sélectionner uniquement les colonnes catégoriques et binaires
    data_categorielles = data[variables_categorielles+variables_binaires]

    # Initialiser l'encodeur pour les variables catégoriques
    preproc_ohe = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

    # Ajuster et transformer les variables catégoriques
    preproc_ohe.fit(data_categorielles)
    variables_categorielles_ohe = preproc_ohe.transform(data_categorielles)

    # Convertir le résultat en DataFrame
    variables_categorielles_ohe = pd.DataFrame(
        variables_categorielles_ohe,
        columns=preproc_ohe.get_feature_names_out(data_categorielles.columns),
        index=data.index  # Conserver les index originaux
    )

    if variables_numeriques:
        data_numeriques = data[variables_numeriques]
        data_numeriques= pd.DataFrame(
            data_numeriques,
            columns=variables_numeriques,
            index=data.index
        )
    else:
        data_numeriques = pd.DataFrame(index=data.index)

    # Supprimer les colonnes catégorielles et binaires d'origine et ajouter les colonnes encodées
    data_final = pd.concat([
        variables_categorielles_ohe,
        data_numeriques
    ], axis=1)

    return data_final


## Random forest


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import pandas as pd
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import ADASYN


In [ ]:


# Encodage des variables catégorielles
y = df['y']
X = pd.get_dummies(df.drop('y', axis=1), drop_first=True)

# Séparation des données en ensembles d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

# Suréchantillonnage des données d'entraînement
sampler = RandomOverSampler(random_state=42)
X_train_resampled, y_train_resampled = sampler.fit_resample(X_train, y_train)

# smote_tomek = SMOTETomek(random_state=42)
# X_train_resampled, y_train_resampled = smote_tomek.fit_resample(X_train, y_train)


# Standardisation des données
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

#### Fitting avec cross validation

In [ ]:
# Recherche des hyperparamètres avec GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 2, 4,6],
    'min_samples_split': [2, 5, 10]
}

num_folds = 3

grid_search = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid,
    scoring='precision',  # Métrique pour maximiser l'AUC-ROC
    cv=KFold(n_splits=num_folds, shuffle=True, random_state=42),
    verbose=2,
    n_jobs=-1
)




In [ ]:
# Initialisation du modèle Random Forest
rf_model = RandomForestClassifier(random_state=42, n_jobs=-1)
rf_model.fit(X_train_resampled, y_train_resampled)

# Prédictions initiales
y_pred = rf_model.predict(X_test)
y_proba = rf_model.predict_proba(X_test)[:, 1]

# Évaluation initiale
print("\n### Matrice de confusion ###")
print(confusion_matrix(y_test, y_pred))

print("\n### Rapport de classification ###")
print(classification_report(y_test, y_pred))

print("\n### Score AUC-ROC ###")
auc_score = roc_auc_score(y_test, y_proba)
print(f"AUC-ROC: {auc_score:.4f}")

In [ ]:
# Ajustement avec GridSearch
grid_search.fit(X_train_resampled, y_train_resampled)

# Résultats de la recherche d'hyperparamètres
print("\n### Meilleurs paramètres ###")
print(grid_search.best_params_)

# Utilisation du meilleur modèle
best_rf_model = grid_search.best_estimator_

# Prédictions finales
final_y_pred = best_rf_model.predict(X_test)
final_y_proba = best_rf_model.predict_proba(X_test)[:, 1]

# Rapport final
print("\n### Rapport final de classification ###")
print(classification_report(y_test, final_y_pred))

print("\n### Score AUC-ROC final ###")
final_auc_score = roc_auc_score(y_test, final_y_proba)
print(f"AUC-ROC final: {final_auc_score:.4f}")

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split, GridSearchCV, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from imblearn.over_sampling import SMOTE
import pandas as pd

# Étape 1 : Encodage des variables catégorielles
y = df['y'].map({'no': 0, 'yes': 1})
X = pd.get_dummies(df.drop('y', axis=1), drop_first=True)

# Étape 2 : Séparation Train/Test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Étape 3 : Suréchantillonnage avec SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Étape 4 : Standardisation des données
scaler = StandardScaler()
X_train_resampled = scaler.fit_transform(X_train_resampled)
X_test = scaler.transform(X_test)

# Étape 5 : Initialisation du modèle Gradient Boosting
gb_model = GradientBoostingClassifier(random_state=42)

# Étape 6 : Entraînement initial sans ajustement d'hyperparamètres
gb_model.fit(X_train_resampled, y_train_resampled)

# Prédictions initiales
y_pred = gb_model.predict(X_test)
y_proba = gb_model.predict_proba(X_test)[:, 1]

# Évaluation initiale
print("\n### Matrice de confusion ###")
print(confusion_matrix(y_test, y_pred))

print("\n### Rapport de classification ###")
print(classification_report(y_test, y_pred))

print("\n### Score AUC-ROC ###")
auc_score = roc_auc_score(y_test, y_proba)
print(f"AUC-ROC: {auc_score:.4f}")

# Étape 7 : Recherche d'hyperparamètres avec GridSearchCV
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

num_folds = 5

grid_search = GridSearchCV(
    estimator=gb_model,
    param_grid=param_grid,
    scoring='roc_auc',
    cv=KFold(n_splits=num_folds, shuffle=True, random_state=42),
    verbose=2,
    n_jobs=-1
)

# Ajustement avec GridSearch
grid_search.fit(X_train_resampled, y_train_resampled)

# Résultats de la recherche d'hyperparamètres
print("\n### Meilleurs paramètres ###")
print(grid_search.best_params_)

# Étape 8 : Utilisation du meilleur modèle
best_gb_model = grid_search.best_estimator_

# Prédictions finales
final_y_pred = best_gb_model.predict(X_test)
final_y_proba = best_gb_model.predict_proba(X_test)[:, 1]

# Rapport final
print("\n### Rapport final de classification ###")
print(classification_report(y_test, final_y_pred))

print("\n### Score AUC-ROC final ###")
final_auc_score = roc_auc_score(y_test, final_y_proba)
print(f"AUC-ROC final: {final_auc_score:.4f}")
